In [6]:
def echange_ligne(A,i, j):
    A[i], A[j] = A[j], A[i]

In [11]:
def transvection(A, i, j, mu):
    L1, L2 = A[i], A[j]
    for k in range(len(L1)):
        L1[k] = L1[k] + mu * L2[k]

In [12]:
A = [[4, 5, 6], [1, 2, 3]]

In [13]:
echange_ligne(A, 0, 1)

In [8]:
A

[[1, 2, 3], [4, 5, 6]]

In [14]:
transvection(A, 1, 0, -4)

In [15]:
A

[[1, 2, 3], [0, -3, -6]]

In [22]:
def pivot_partiel(A, j0):
    imax, cmax = j0, abs(A[j0][j0])
    for j in range(j0 + 1, len(A)):
        c = abs(A[j][j0])
        if c > cmax:
            imax, cmax = j, c
    return imax    

In [18]:
A = [[1, 2, 3, 4], [0, 1, 3, 5], [0, -4, 1, 0], [0, 3, 0, 0]]

In [23]:
pivot_partiel(A, 1)

2

In [48]:
def resolution_systeme2(A0, y0):
    #copie de A0 matrice augmentée de la colonne y0
    n = len(A0)
    A = [ A0[k][:] + y0[k] for k in range(n)]    
    #A0 supposée de Cramer avec donc n pivots
    for i in range(n - 1):
        j = pivot_partiel(A, i)       #recherche de la ligne du pivot partiel
        echange_ligne(A, i, j)  
        pivot = A[i][i]
        #tranvections pour mettre des zeros dans la colonne en-dessous du pivot
        for k in range(i + 1, n):
            transvection(A, k, i, - A[k][i]/pivot)
    #phase de remontée
    x = [[0] for _ in range(n)]
    for i in range(n - 1, -1, -1):
        x[i] = [(A[i][-1] - sum(A[i][j] * x[j][0] for j in range(i + 1, n)))/ A[i][i]]
    return x

In [47]:
resolution_systeme([[2, 3],[5,-2]],[[5], [-16]])

[[-2.0], [3.0000000000000004]]

In [133]:
import numpy as np

def inversion_matrice2(A0):
    #Copie de A0 augmentée de la matrice identité
    n = len(A0)    
    A = np.array([A0[k][:] + [ 1 if j == k else 0 for j in range(n)]
                  for k in range(len(A0))], dtype='float64')
    for i in range(n - 1):
        j = pivot_partiel(A, i)       #recherche de la ligne du pivot partiel
        echange_ligne(A, i, j)  
        pivot = A[i][i]
        #transvections
        for k in range(i + 1, n):
            A[k] -= A[i] * (A[k][i] /pivot)
    #phase de remontée    
    for i in range(n - 1, -1, -1): 
        A[i] =  (1/A[i][i]) * (A[i] - sum(A[j] * A[i][j] for j in range(i + 1, n)))       
    return A[:,n:]

In [53]:
A3 = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
A4 = [[1, 1./4, 1], [1, 1./3, 2], [0, 1, 12]]
A5 = [[1,1.+10**5,1],[1,1+10**(-5),2],[0,10**5,-1]]
A6 = [[1,10**15,1],[1,10**(-2),2],[0,10**15,-1]]

In [130]:
inversion_matrice2([[1,2],[0,3]])

array([[ 1.        , -0.66666667],
       [ 0.        ,  0.33333333]])

In [134]:
inversion_matrice2([[2,2,-3],[0,1,-6],[0,0,1]])

array([[ 0.5, -1. , -4.5],
       [ 0. ,  1. ,  6. ],
       [ 0. ,  0. ,  1. ]])

In [1]:
def echange_ligne(A, i, j):
    """Il suffit d'échanger les pointeurs"""
    A[i], A[j] = A[j], A[i]
    
def pivot_partiel(A, j0):
    """Recherche du pivot de module maximal dans la colonne j0 de
    la matrice A, parmi les lignes d'index >= j0"""
    i = j0 #ligne du maximum provisoire
    modulepivot = abs(A[i][j0])
    for k in range(j0+1, len(A)):
        if abs(A[k][j0]) > modulepivot:
            i, modulepivot = k, abs(A[k][j0])
    return i

def transvection(A, i, j, mu):
    """Transvection Li <- Li + mu*Lj"""
    for k in range(len(A[i])):
        A[i][k] += mu*A[j][k]
        
def copie_matrice(M):
    return [ligne[:] for ligne in M]
    
def dilatation(A, i, mu):
    """Li <- Li*mu si mu non nul"""
    assert mu != 0
    #A[i] = [A[i][k]*mu for k in range(len(A[i])]
    for k in range(len(A[i])):
        A[i][k] *= mu

def resolution4(A0, y0):
    n = len(A0)
    A = [ A0[k][:] + y0[k][:] for k in range(n)] 
    for i in range(n - 1):
        j = pivot_partiel(A, i)  #recherche de la ligne du pivot partiel
        echange_ligne(A, i, j)
        pivot = A[i][i]
        for k in range(i + 1, n):
            transvection(A, k, i, - A[k][i]/pivot)
    for i in range(n - 1, -1, -1):
        for j in range(n - 1, i, -1):
            transvection(A, i, j, -A[i][j]/A[j][j])
        dilatation(A, i, 1/A[i][i])
    return [A[k][-1] for k in range(n)]
            
        
def inverse4(A0):
    n = len(A0)
    A = [ A0[k][:] + [ 1 if j == k else 0 for j in range(n)]
                  for k in range(n)]
    for i in range(n - 1):
        j = pivot_partiel(A, i)  #recherche de la ligne du pivot partiel
        echange_ligne(A, i, j)
        pivot = A[i][i]
        for k in range(i + 1, n):
            transvection(A, k, i, - A[k][i]/pivot)
    for i in range(n - 1, -1, -1):
        for j in range(n - 1, i, -1):
            transvection(A, i, j, -A[i][j]/A[j][j])
        dilatation(A, i, 1/A[i][i])
    return A

In [2]:
def pivot_gauss(B):
    """Pivot de Gauss appliqué à une matrice B"""
    n = len(B)
    A = [ B[k][:]  for k in range(n)]  #Copie de B
    for i in range(n - 1):
        j = pivot_partiel(A, i)  #recherche de la ligne du pivot partiel
        echange_ligne(A, i, j)
        pivot = A[i][i]
        for k in range(i + 1, n):
            transvection(A, k, i, - A[k][i]/pivot)
    for i in range(n - 1, -1, -1):
        for j in range(i + 1, n, 1):
            transvection(A, i, j, -A[i][j]/A[j][j])
        dilatation(A, i, 1/A[i][i])
    return A            

In [3]:
def resolution_systeme2(A0, y0):
    matricePlus = [ A0[k][:] + y0[k][:] for k in range(len(A0))] 
    A = pivot_gauss(matricePlus)
    return [Ligne[-1] for Ligne in A]

def inversion2(A0):
    n = len(A0)
    matricePlus = [ A0[k][:] + [ 1 if j == k else 0 for j in range(n)] for k in range(n)]
    A = pivot_gauss(matricePlus)
    return [Ligne[n:] for Ligne in A]

In [4]:
 resolution_systeme2([[2, 3], [5, -2]], [[5], [-16]])

[-2.0, 3.0]

In [6]:
inversion2(([[2,2,-3],[-2,-1,-3],[6,4,4]]))

[[4.000000000000008, -10.000000000000021, -4.500000000000009],
 [-5.000000000000011, 13.000000000000027, 6.000000000000012],
 [-1.0000000000000018, 2.0000000000000044, 1.000000000000002]]